In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv('/kaggle/input/playground-series-s3e17/train.csv')
x=pd.read_csv('/kaggle/input/machine-failure-predictions/machine failure.csv')
x=x.drop(columns=['UDI'])
train=pd.concat([train,x],ignore_index=True)
test=pd.read_csv('/kaggle/input/playground-series-s3e17/test.csv')
print(train.shape)
train.head()

In [ ]:
train=train.drop(['id'],axis=1)
train

In [ ]:
list1=[]
list2=[]
list3=[]
list4=[]
for i in train['Product ID']:
    c=int(i[1::])
    x=c+ord(i[0])
    list1.append(x)
for i in test['Product ID']:
    c=int(i[1::])
    x=c+ord(i[0])
    list2.append(x)
train['Product ID']=list1
test['Product ID']=list2
for i in train['Type']: 
    x=ord(i)
    list3.append(x)
for i in test['Type']:
    x=ord(i)
    list4.append(x)
train['Type']=list3
test['Type']=list4

In [ ]:
train.head()

In [ ]:
Y=train['Machine failure']
train=train.drop(['Machine failure'],axis=1)
train.head()

In [ ]:
Sum1=train.sum()
Sum2=test.sum()

In [ ]:
import math as m
stan_dev_col1=[]
for i in train:
    stan_dev=[]
    for j in train[i]:
        z=j-Sum1[i]/train.shape[0]
        z=z*z
        stan_dev.append(z)
    summ=sum(stan_dev)
    stan_dev_col1.append(m.sqrt(summ/train.shape[0]))
stan_dev_col2=[]
for i in test:
    stan_dev=[]
    for j in test[i]:
        z=j-Sum2[i]/test.shape[0]
        z=z*z
        stan_dev.append(z)
    summ=sum(stan_dev)
    stan_dev_col2.append(m.sqrt(summ/test.shape[0]))


In [ ]:
count =0
for i in train:
    Z_Nor1=[]
    for j in train[i]:
        mean=(Sum1[count]/train.shape[0])
        z=(j-mean)/stan_dev_col1[count]
        Z_Nor1.append(z)
    train[i]=Z_Nor1
    count+=1
count=0        
for i in test:
    Z_Nor2=[]
    for j in test[i]:
        mean=(Sum2[count]/test.shape[0])
        z=(j-mean)/stan_dev_col2[count]
        Z_Nor2.append(z)
    test[i]=Z_Nor2
    count+=1

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import scipy.stats as st

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
param_grid ={'max_depth': st.randint(6, 11),
             'n_estimators':st.randint(300, 500),
             'max_features':np.arange(0.5,.81, 0.05),
            'max_leaf_nodes':st.randint(6, 10)}

grid = RandomizedSearchCV(rfc,
                    param_grid, cv=10,
                    scoring='accuracy',
                    verbose=1,n_iter=20)

grid.fit(train, Y)

In [ ]:
sub = test[['id']]
test=test.drop(['id'],axis=1)

In [ ]:
y_prob = grid.predict_proba(test)

In [ ]:
sub['Machine failure'] = y_prob[:,1]

In [ ]:
sub

In [ ]:
sub.to_csv("submissions_cat_2.csv", index=False)